In [1]:
#import the necessary libraries
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
import scipy.spatial.distance as scd
from sklearn.metrics import jaccard_similarity_score

pd.set_option("display.max_rows", 16)
LARGE_FIGSIZE = (20, 10)

In [2]:
#change directory
#%cd ~/cse_601/project2/
file_name = "iyer.txt"
#file_name = "kmean_ex2.txt"
#initialize number of clusters 
num_clusters = 11
#cluster_indexes = [1,2]
cluster_indexes = list()

In [3]:
#access and store the required data
input_df = pd.read_table(file_name,header=None)
#input_df

#rename and reorganize data
input_df.rename(columns={input_df.columns[0]:'gene_id'}, inplace=True)
input_df.rename(columns={input_df.columns[1]:'gnd_truth_cluster'}, inplace=True)
input_df = input_df.set_index(input_df.columns[0])
#input_df

#create a copy of original data and drop required columns
gdf = input_df.copy()
gdf.drop('gnd_truth_cluster',axis=1,inplace=True)
data_attribute_list = list(gdf.columns)

#reference array for external index calculation
gnd_truth = np.array(input_df.loc[:,'gnd_truth_cluster'],dtype=int)
gnd_truth_np = np.array(input_df.loc[:,'gnd_truth_cluster'],dtype=int)

#create two lists - one for cluster names (adding new rows) and another for storing distance from the clusters(cols)
str1 = "cluster_"
str2 = "dist_"
cluster_list = []
cluster_dist_list = []
for i in range(1,num_clusters+1):
        cluster_list.append(str1+str(i))
        cluster_dist_list.append(str2 + str1+str(i))
#print(cluster_list)
#print(cluster_dist_list)

#create list of sets(to compare changes in clustering over iterations)
prev_list_set_clusters = list()
cur_list_set_clusters = list()

#initialize clusters
import random
if(len(cluster_indexes)==0):
    cluster_indexes = random.sample(input_df.index.tolist(), num_clusters)
    #cluster_indexes = random.sample(xrange(1,len(gdf.index)), num_clusters)
    
#inserting the clusters as rows
for i in range(0,len(cluster_list)):
        gdf.loc[cluster_list[i]] = gdf.loc[cluster_indexes[i]]

#create columns to store the distance of genes from each cluster
for i in range(0,len(cluster_dist_list)):
    gdf[cluster_dist_list[i]] = 0

def k_means_terminate_condition(prev_set_list, cur_set_list):
    #print(len(prev_set_list))
    #print(len(cur_set_list))
    if(len(prev_set_list)==0):
        return True
    else:
        flag = True
        for j in range(0,len(cur_set_list)):
            flag = (flag and (set(prev_set_list[j])==set(cur_set_list[j])))
            #flag = flag and ((prev_set_list[j].issubset(cur_set_list[j])) and (cur_set_list[j].issubset(prev_set_list[j])))
        return not flag
            

prev_list_set_clusters=[]
cur_list_set_clusters = []
num_iter = 0
attr_cnt = len(data_attribute_list)
#terminate the k-means algorithm when there is no difference in clusters between two iterations
while(k_means_terminate_condition(prev_set_list=prev_list_set_clusters, cur_set_list=cur_list_set_clusters)):
    num_iter += 1
    if(len(cur_list_set_clusters)>0):
        prev_list_set_clusters = cur_list_set_clusters
        cur_list_set_clusters = []
    #calculate euclidean distance of each gene_id from each cluster centroid
    for i in range(0,len(cluster_list)):
        gdf[cluster_dist_list[i]]=gdf.apply((lambda x:scd.euclidean(x[0:attr_cnt],gdf.iloc[gdf.index.get_loc(cluster_list[i]),0:attr_cnt])),axis=1)
    #assign the least distance of gene_id from cluster centroids to cluster columns
    gdf['cluster']=gdf.loc[:,cluster_dist_list].idxmin(axis=1)
    #string manipulation - to get the cluster_name assigned to each gene_id
    gdf['cluster']=gdf['cluster'].map(lambda x:str(x).strip("dist_"))
    #populate list of indexes(gene_ids) for a particular cluster
    for i in range(0,len(cluster_list)):
        cur_list_set_clusters.append(set(gdf.index[gdf['cluster']==cluster_list[i]].tolist()[:-1]))
        gdf.iloc[gdf.index.get_loc(cluster_list[i]),0:attr_cnt]= gdf.loc[(gdf['cluster']==cluster_list[i]),:].iloc[:,0:attr_cnt].mean(axis=0)

#renaming the values in cluster column to integers
for i in range(0,len(cur_list_set_clusters)):
    gdf.loc[cur_list_set_clusters[i],'cluster'] = i+1

#predicted clustering by our implementation
pred = np.array(gdf.loc[input_df.index,'cluster'],dtype=int)
pred_original = pred[:]

In [4]:
#purpose of below code - generating different combinations of cluster names and checking for jaccard co efficient
#ex. the same cluster in ground truth maybe called as "2" maybe called "3" in generated cluster
# if pts [a:d] had ground truth mentioned as [1,2,1,2] had cluster_name generated as [2,1,1,2]
# then in reality the same cluster of points are called by different names
import itertools
def label_perm(x):
    idx = x-1
    return perm_list[idx]
def get_jc_label_permute(max_cluster_label_num,predicted_label,gnd_truth_vector):
    # returns final_label_pred,jc_value
    init_label = predicted_label[:]
    max_jc_value = 0
    final_label_pred = list()
    perm_list_cluster_labels = list(itertools.permutations(range(1,max_cluster_label_num+1)))
    iter_cnt = 0
    for tuple_labels in perm_list_cluster_labels:
        if(iter_cnt<25):
            iter_cnt+=1
            perm_cluster_label_list = list(tuple_labels)
            #pred = map(label_perm),init_label)
            pred = map((lambda x: perm_cluster_label_list[x-1]),init_label)
            js_score = jaccard_similarity_score(gnd_truth_vector,pred)
            if(js_score>max_jc_value):
                max_jc_value = js_score
                final_label_pred = pred[:]
        else:
            break
    return (max_jc_value,final_label_pred)

In [5]:
#get label_map for the max j_score
j_score,pred_label_map = get_jc_label_permute(max_cluster_label_num = 11,predicted_label=pred ,gnd_truth_vector= gnd_truth_np)

TypeError: Expected sequence or array-like, got <class 'map'>

In [ ]:
j_score

In [ ]:
#predicted clustering
pred

In [ ]:
#original clustering
np.array(gdf.cluster)

In [ ]:
data_attribute_list

In [ ]:
cluster_indexes

In [ ]:
#input_df.index.tolist()

In [ ]:
#cho.txt
#[362, 295, 243, 133, 383] -- 0.47
#[55, 23, 198, 358, 207] -- 0.349
#iyer.txt -- presence of big number of outliers reduces the performance of the algorithm
#[472, 289, 434, 264, 325, 18, 47, 17, 19, 378] -- 0.15

In [ ]:
#getting indices of columns with data
data_attribute_list
col_list = list()
for e in data_attribute_list:
    col_list.append(gdf.columns.get_loc(e))
col_list

In [ ]:
# Package to perform PCA
import sklearn.datasets
import sklearn.decomposition

# Perform the PCA again retaining only the top 2 components
sklearn_pca = sklearn.decomposition.PCA(n_components=2)
sklearn_pca.fit(np.array(gdf.loc[input_df.index.tolist(),col_list]))

# Project the data into this 2D space and convert it back to a tidy dataframe
df_2D = pd.DataFrame(sklearn_pca.transform(np.array(gdf.loc[input_df.index.tolist(),col_list])),
                     columns=['PCA1', 'PCA2'])
df_2D['clusters'] = gdf['cluster']
df_2D.head()
for key, group in df_2D.groupby(['clusters']):
    plt.plot(group.PCA1, group.PCA2, 'o', alpha=0.7, label=key)

# Tidy up plot
plt.legend(loc=0, fontsize=15)
plt.margins(0.05)
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')

comparing against sci kit kmeans

X= np.array(input_df.iloc[:,1:])
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5)
kmeans.fit(X)
centroids = kmeans.cluster_centers_
labels = kmeans.labels_
labels = labels+1

jaccard_similarity_score(gnd_truth_np,labels)

gnd_truth

X.shape